In [1]:
import os, sys
import numpy as np
import torch
import json

sys.path.append("..")
from singleVis.SingleVisualizationModel import VisModel
from singleVis.data import NormalDataProvider
from singleVis.eval.evaluator import Evaluator
from singleVis.projector import Projector, tfDVIProjector, TimeVisProjector

In [2]:
# tensorflow
visible_device = "0,1,2,3"
os.environ["CUDA_VISIBLE_DEVICES"] = visible_device

In [3]:
dataset = "mnist"
noise_type = "symmetric"
noise_rate = "5"

In [4]:
VIS_METHOD = "TimeVis"
CONTENT_PATH = "/home/xianglin/projects/DVI_data/noisy/{}/{}/{}/".format(noise_type, dataset, noise_rate)
sys.path.append(CONTENT_PATH)
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

In [5]:
CLASSES = config["CLASSES"]
GPU_ID = config["GPU"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]


# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]
# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
VIS_MODEL_NAME = VISUALIZATION_PARAMETER["VIS_MODEL_NAME"]

TOTOAL_EPOCH = (EPOCH_END-EPOCH_START)//EPOCH_PERIOD + 1

# define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

import Model.model as subject_model
# net = resnet18()
net = eval("subject_model.{}()".format(NET))

data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, device=DEVICE, classes=CLASSES, epoch_name="Epoch", verbose=1)

if VIS_METHOD == "tfDVI":
    # Define Projector
    flag = "_temporal_id_withoutB"
    projector = tfDVIProjector(CONTENT_PATH, flag=flag)
elif VIS_METHOD == "TimeVis":
    model = VisModel(ENCODER_DIMS, DECODER_DIMS)
    projector = TimeVisProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, device=DEVICE)
elif VIS_METHOD == "DeepDebugger":
    model = VisModel(ENCODER_DIMS, DECODER_DIMS)
    SEGMENTS = VISUALIZATION_PARAMETER["SEGMENTS"]
    projector = Projector(vis_model=model, content_path=CONTENT_PATH, segments=SEGMENTS, device=DEVICE)

Finish initialization...


In [6]:
import pickle
with open(os.path.join(CONTENT_PATH,  '{}_sample_recommender.pkl'.format(VIS_METHOD)), 'rb') as f:
    tm = pickle.load(f)

In [6]:
# get images
import torch
import torchvision
from torch.utils.data import DataLoader
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('data/', train=True, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=1, shuffle=True)

In [7]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('data/', train=True, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=1, shuffle=True)

In [54]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.SVHN(
    root='data',
    split='train',
    download=True,
    transform=torchvision.transforms.ToTensor()),
    batch_size=1, shuffle=True)


  0%|          | 0/182040794 [00:00<?, ?it/s]

In [46]:
# true sample
samples = np.zeros((TOTOAL_EPOCH, 100, 512))
for i in range(EPOCH_START, EPOCH_END, EPOCH_PERIOD):
    e = (i-EPOCH_START)//EPOCH_PERIOD
    samples[e] = data_provider.test_representation(i)[:100]

embeddings_2d = np.zeros((TOTOAL_EPOCH, 100, 2))
for e in range(1, TOTOAL_EPOCH+1, 1):
    embeddings_2d[e-1] = projector.batch_project(e, samples[e-1])
embeddings_2d = np.transpose(embeddings_2d, [1,0,2])

for i in range(100):
    print(tm.score_new_sample(embeddings_2d[i][-tm.period:]))

(0.96792, 0.91814, 0.99998)
(0.96944, 0.92994, 0.89802)
(0.93274, 0.92994, 0.89802)
(0.96658, 0.92604, 0.98538)
(0.95586, 0.91968, 0.85686)
(0.96598, 0.91968, 0.85686)
(0.96294, 0.90114, 0.98758)
(0.95586, 0.98546, 0.98382)
(0.97498, 0.91814, 0.99998)
(0.96598, 0.92994, 0.86044)
(0.9795, 0.92604, 0.88068)
(0.96064, 0.92972, 0.93146)
(0.95788, 0.93022, 0.85686)
(0.9314, 0.91814, 0.92838)
(0.96064, 0.92972, 0.93146)
(0.96944, 0.92994, 0.89802)
(0.95788, 0.93022, 0.85686)
(0.97858, 0.91814, 0.92838)
(0.96944, 0.92994, 0.88068)
(0.95586, 0.91968, 0.92838)
(0.9314, 0.91814, 0.99074)
(0.9589, 0.92604, 0.88068)
(0.9365600000000001, 0.93022, 0.94854)
(0.96064, 0.92972, 0.93146)
(0.9801, 0.963, 0.99998)
(0.9399, 0.98546, 0.98382)
(0.9365600000000001, 0.93022, 0.85686)
(0.9589, 0.92604, 0.88068)
(0.96064, 0.92972, 0.93146)
(0.95586, 0.91814, 0.85686)
(0.95586, 0.91968, 0.85686)
(0.95788, 0.93022, 0.85686)
(0.9365600000000001, 0.93022, 0.98696)
(0.9801, 0.93022, 0.85686)


KeyboardInterrupt: 

In [48]:
# mixup
samples = np.zeros((TOTOAL_EPOCH, LEN, 512))
for i in range(EPOCH_START, EPOCH_END, EPOCH_PERIOD):
    e = (i-EPOCH_START)//EPOCH_PERIOD
    samples[e] = data_provider.train_representation(i)
labels = data_provider.train_labels(20)

for i in range(10):
    while True:
        idx1, idx2 = np.random.choice(LEN, 2, replace=False)
        if labels[idx1]!=labels[idx2]:
            break

    mixup = 0.5*samples[:,idx1,:]+0.5*samples[:,idx2,:]+0.5
    embedding_2d = np.zeros((TOTOAL_EPOCH,2))
    for e in range(1, TOTOAL_EPOCH+1, 1):
        embedding_2d[e-1] = projector.batch_project(e, mixup[np.newaxis,e-1])[0]
    print(tm.score_new_sample(embedding_2d[-tm.period:]))

(0.94708, 0.92972, 0.93146)
(0.97934, 0.91814, 0.92838)
(0.96598, 0.92972, 0.98758)
(0.9365600000000001, 0.93022, 0.85686)
(0.99088, 0.9858, 0.85686)
(0.94708, 0.91968, 0.97754)
(0.94708, 0.91968, 0.93146)
(0.95586, 0.91968, 0.99998)


KeyboardInterrupt: 

In [10]:
# OOD
C=0
for img, target in train_loader:
    C = C+1
    image = img.detach().cpu().numpy()
    # image = np.pad(image, ((0,0),(1,1), (2, 2), (2, 2)), 'edge')
    image = image[:, :1, :28,:28]
    embedding_2d = np.zeros((TOTOAL_EPOCH, 2))
    for i in range(EPOCH_START, EPOCH_END, EPOCH_PERIOD):
        e = int((i-EPOCH_START)/EPOCH_PERIOD)
        repr = data_provider.feature_function(i)(torch.from_numpy(image).to(DEVICE))
        embedding_2d[e] = projector.batch_project(i, repr.detach().cpu().numpy())
    print(tm.score_new_sample(embedding_2d[-tm.period:]))
    if C ==100:
        break

(0.9162833333333333, 0.9985833333333334, 0.9972666666666666)
(0.9999833333333333, 0.7139333333333333, 0.9982666666666666)
(0.9982666666666666, 0.908, 0.9071)
(0.9982666666666666, 0.9982666666666666, 0.9071)
(0.9994666666666666, 0.9976333333333334, 0.9982666666666666)
(0.9162833333333333, 0.7139333333333333, 0.7149333333333333)
(0.9012166666666667, 0.6979333333333333, 0.9015)


KeyboardInterrupt: 